In [6]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from datetime import datetime

In [7]:
"https://app.enappsys.com/datadownload?code=hu/elec/pricing/daprices&currency=EUR&delimiter=comma&minavmax=false&pass=204245237229238244242225228229178224178181&res=qh&tag=csv&timezone=CET&user=peter.koleszar@solservices.hu&start=202501130000&end=202510190000"

'https://app.enappsys.com/datadownload?code=hu/elec/pricing/daprices&currency=EUR&delimiter=comma&minavmax=false&pass=204245237229238244242225228229178224178181&res=qh&tag=csv&timezone=CET&user=peter.koleszar@solservices.hu&start=202501130000&end=202510190000'

In [8]:
import io
from datetime import datetime, timedelta

# Define one week from each season in 2025
# January (winter), April (spring), June (summer), October (fall)
weeks_2025 = [
    ("202501080000", "202501150000"),  # January week (Jan 8-15)
    ("202504080000", "202504150000"),  # April week (Apr 8-15)
    ("202506100000", "202506170000"),  # June week (Jun 10-17)
    ("202510070000", "202510140000"),  # October week (Oct 7-14)
]

API_KEY = "204245237229238244242225228229178176178181"
BASE_URL = "https://app.enappsys.com/datadownload"

all_dfs = []
for i, (start, end) in enumerate(weeks_2025, 1):
    url = (
        f"{BASE_URL}?code=hu/elec/pricing/daprices"
        f"&currency=EUR"
        f"&delimiter=comma"
        f"&minavmax=false"
        f"&pass={API_KEY}"
        f"&res=hourly"
        f"&tag=csv"
        f"&timezone=CET"
        f"&user=peter.koleszar@solservices.hu"
        f"&start={start}"
        f"&end={end}"
    )
    
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()
    
    df_week = pd.read_csv(io.StringIO(resp.text))
    all_dfs.append(df_week)
    print(f"Week {i}/4: {start[:8]} to {end[:8]} -> {len(df_week)} rows")

# Concatenate all weeks
montel_df = pd.concat(all_dfs, ignore_index=True)
print(f"\nTotal downloaded: {len(montel_df)} rows from 4 weeks across 2024 seasons")


Week 1/4: 20250108 to 20250115 -> 169 rows
Week 2/4: 20250408 to 20250415 -> 169 rows
Week 3/4: 20250610 to 20250617 -> 169 rows
Week 4/4: 20251007 to 20251014 -> 169 rows

Total downloaded: 676 rows from 4 weeks across 2024 seasons


In [9]:
montel_df

,Date (CET),AUSTRIA (AT),CROATIA (HR),GERMANY (DE),HUNGARY (HU),ROMANIA (RO),SERBIA (RS),SLOVAKIA (SK),SLOVENIA (SL)
0,NaN,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh,EUR/MWh
1,[08/01/2025 00:00],84.74,83.48,71.05,84.48,83.60,78.57,85.10,83.43
2,[08/01/2025 01:00],84.87,75.34,68.01,75.07,74.64,69.56,74.81,76.02
3,[08/01/2025 02:00],87.45,73.06,63.34,72.94,72.07,71.67,72.83,74.03
4,[08/01/2025 03:00],80.00,69.17,57.01,69.99,68.96,65.07,70.87,69.67
...,...,...,...,...,...,...,...,...,...
671,[13/10/2025 19:00],301.58,331.15,306.58,394.32,379.99,372.00,271.62,315.07
672,[13/10/2025 20:00],191.56,245.64,186.95,336.14,318.01,250.07,123.50,221.47
673,[13/10/2025 21:00],141.54,177.64,137.18,233.39,225.27,174.94,96.44,162.26
674,[13/10/2025 22:00],127.01,132.97,121.99,139.37,138.71,134.46,119.05,131.09


In [10]:
montel_df = montel_df[["HUNGARY (HU)", "Date (CET)"]]

In [11]:
montel_df

,HUNGARY (HU),Date (CET)
0,EUR/MWh,NaN
1,84.48,[08/01/2025 00:00]
2,75.07,[08/01/2025 01:00]
3,72.94,[08/01/2025 02:00]
4,69.99,[08/01/2025 03:00]
...,...,...
671,394.32,[13/10/2025 19:00]
672,336.14,[13/10/2025 20:00]
673,233.39,[13/10/2025 21:00]
674,139.37,[13/10/2025 22:00]


In [12]:
# Inspect the date format first
print("Sample date values:")
print(montel_df["Date (CET)"].head())
print(f"\nFirst date type: {type(montel_df['Date (CET)'].iloc[0])}")
print(f"First date value: {repr(montel_df['Date (CET)'].iloc[0])}")

# Convert date column to datetime (strip brackets if present)
montel_df["Date (CET)"] = montel_df["Date (CET)"].astype(str).str.strip("[]")
montel_df["Date (CET)"] = pd.to_datetime(montel_df["Date (CET)"], format="%d/%m/%Y %H:%M", errors="coerce")

print(f"\nConverted date column to datetime. Date range: {montel_df['Date (CET)'].min()} to {montel_df['Date (CET)'].max()}")
print(f"Failed conversions: {montel_df['Date (CET)'].isna().sum()}")


Sample date values:
0                   NaN
1    [08/01/2025 00:00]
2    [08/01/2025 01:00]
3    [08/01/2025 02:00]
4    [08/01/2025 03:00]
Name: Date (CET), dtype: object

First date type: <class 'float'>
First date value: nan

Converted date column to datetime. Date range: 2025-01-08 00:00:00 to 2025-10-13 23:00:00
Failed conversions: 4


C:\Users\Baris Sever\AppData\Local\Temp\ipykernel_13304\4248325296.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montel_df["Date (CET)"] = montel_df["Date (CET)"].astype(str).str.strip("[]")
C:\Users\Baris Sever\AppData\Local\Temp\ipykernel_13304\4248325296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montel_df["Date (CET)"] = pd.to_datetime(montel_df["Date (CET)"], format="%d/%m/%Y %H:%M", errors="coerce")


In [13]:
# Convert price column to float and set date as index
montel_df["HUNGARY (HU)"] = pd.to_numeric(montel_df["HUNGARY (HU)"], errors="coerce")
montel_df = montel_df.set_index("Date (CET)", drop=True)

print(f"Price column converted to float. Data shape: {montel_df.shape}")
print(f"Index is datetime: {isinstance(montel_df.index, pd.DatetimeIndex)}")
print(f"\nSample data:")
print(montel_df.head())


Price column converted to float. Data shape: (676, 1)
Index is datetime: True

Sample data:
                     HUNGARY (HU)
Date (CET)                       
NaT                           NaN
2025-01-08 00:00:00         84.48
2025-01-08 01:00:00         75.07
2025-01-08 02:00:00         72.94
2025-01-08 03:00:00         69.99


C:\Users\Baris Sever\AppData\Local\Temp\ipykernel_13304\4181801999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  montel_df["HUNGARY (HU)"] = pd.to_numeric(montel_df["HUNGARY (HU)"], errors="coerce")


In [14]:
# Drop all rows where price is NaN
initial_count = len(montel_df)
nan_count = montel_df["HUNGARY (HU)"].isna().sum()
montel_df = montel_df.dropna(subset=["HUNGARY (HU)"])
final_count = len(montel_df)

print(f"Dropped {initial_count - final_count} rows with NaN prices")
print(f"Remaining rows: {final_count}")
print(f"NaN values remaining: {montel_df['HUNGARY (HU)'].isna().sum()}")


Dropped 4 rows with NaN prices
Remaining rows: 672
NaN values remaining: 0
